In [1]:
# hava a look version
import torch
# import tensorflow as tf
# torch.__version__,tf.__version__
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')#torch.device('cpu')#
print(device,torch.__version__)
# print(device,torch.__version__,tf.__version__)

cpu 1.8.0+cpu


# intent0模型训练

In [3]:
import os
import time
import gensim
import random
import numpy as np
import pickle as pk
import pandas as pd
from tqdm import tqdm
from operator import itemgetter
from collections import defaultdict
# from sklearn.cross_validation import train_test_split
# import sklearn.model_selection


# 准备好模型的参数
parameter = {
    'min_count_word':1,
    'char2ind':None,
    'ind2char':None,
    'ind2embeding':None,
    'output_size': None,
    'epoch':300,
    'batch_size':100,
    'embedding_dim':300,
    'hidden_size':128,
    'num_layers':2, #堆叠LSTM的层数，默认值为1
    'dropout':0.5,
    'cuda':device,
    'lr':0.001,
    'num_unknow':0
}

def build_dataSet(data,parameter):
    # 构建训练集（chars，labels），构建词汇表（char2ind，ind2char）
    chars = []
    labels = []
    vocab = defaultdict(int)
    vocab['<pad>'] = parameter['min_count_word']
    for text,label in tqdm(zip(data.text,data.label)):
        chars.append(text)
        labels.append(label)
        for char in chars[-1]:
            vocab[char] += 1
    vocab['<unk>'] = parameter['min_count_word']
    for char in vocab:
        if vocab[char] < parameter['min_count_word']:
            del vocab[char]
    char2ind,ind2char = dict(zip(vocab.keys(),range(len(vocab)))), \
    dict(zip(range(len(vocab)),vocab.keys()))
    ind2embeding = np.random.randn(len(vocab), parameter['embedding_dim']).astype(np.float32) / np.sqrt(len(vocab))
    # 加载词向量
    w2v = gensim.models.Word2Vec.load('data/wiki.Mode')
    for ind,i in enumerate(char2ind.keys()):
        try:
            embedding = np.asarray(w2v.wv[i], dtype='float32')
            ind2embeding[ind] = embedding
        except:
            parameter['num_unknow'] += 1
    parameter['ind2char'] = ind2char
    parameter['char2ind'] = char2ind
    parameter['ind2embeding'] = ind2embeding
    parameter['output_size'] = len(set(labels))
    return np.array(chars),np.array(labels)

def batch_yield(chars,labels,parameter,shuffle = True):
    for train_epoch in range(parameter['epoch']):
        if shuffle:
            permutation = np.random.permutation(len(chars))
            chars = chars[permutation]
            labels = labels[permutation]
        max_len = 0
        batch_x,batch_y = [],[]
        for iters in tqdm(range(len(chars))):
            batch_ids = itemgetter(*chars[iters])(parameter['char2ind'])
            try:
                batch_ids = list(batch_ids)
            except:
                batch_ids = [batch_ids,0]
            if len(batch_ids) > max_len:
                max_len = len(batch_ids)
            batch_x.append(batch_ids)
            batch_y.append(labels[iters])
            if len(batch_x) >= parameter['batch_size']:
                batch_x = [np.array(list(itemgetter(*x_ids)(parameter['ind2embeding']))+[parameter['ind2embeding'][0]]*(max_len-len(x_ids))) for x_ids in batch_x]
                device = parameter['cuda']
                yield torch.from_numpy(np.array(batch_x)).to(device),torch.from_numpy(np.array(batch_y)).to(device).long(),True,None
                max_len,batch_x,batch_y = 0,[],[]
        batch_x = [np.array(list(itemgetter(*x_ids)(parameter['ind2embeding']))+[parameter['ind2embeding'][0]]*(max_len-len(x_ids))) for x_ids in batch_x]
        device = parameter['cuda']
        yield torch.from_numpy(np.array(batch_x)).to(device),torch.from_numpy(np.array(batch_y)).to(device).long(),True,train_epoch
        max_len,batch_x,batch_y = 0,[],[]
    yield None,None,False,None
    
def batch_yield_predict(chars,parameter):
    batch_x,batch_y = [],[]
    for iters in range(len(chars)):
        if chars[iters] in parameter['char2ind']:
            batch_x.append(parameter['ind2embeding'][parameter['char2ind'][chars[iters]]])
        else:
            batch_x.append(parameter['ind2embeding'][parameter['char2ind']['<unk>']])
    batch_x = [batch_x]
    device = parameter['cuda']
    return torch.from_numpy(np.array(batch_x)).to(device)#,torch.from_numpy(np.array(batch_y)).to(device).long()


[x_intent,y_intent] = pk.load(open('data/data-intent0.pkl','rb'))
data = pd.DataFrame(np.array([x_intent,y_intent]).transpose())
data.columns = ['text','label']
train_chars,train_labels = build_dataSet(data,parameter)

train_yield = batch_yield(train_chars,train_labels,parameter)
seqs,label,keys,epoch = next(train_yield)
seqs,labels,_,_ = next(train_yield)
parameter['model_path'] = 'model/intent0/'
os.mkdir(parameter['model_path']) if not os.path.exists(parameter['model_path']) else 1

pk.dump(parameter,open(parameter['model_path']+'parameter.pkl','wb'))
seqs.shape,parameter['num_unknow']

D:\Anaconda3-5.3\lib\site-packages\ipykernel_launcher.py:107: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
532it [00:00, 512605.04it/s]
D:\Anaconda3-5.3\lib\site-packages\ipykernel_launcher.py:63: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|                                                                                          | 0/532 [00:00<?, ?it/s]

(torch.Size([100, 15, 300]), 9)

In [4]:
# 校验一下
parameter['ind2char'] 
parameter['char2ind'] 
parameter['ind2embeding'].shape
parameter['output_size'],parameter['model_path']

(3, 'model/intent0/')

In [5]:
import torch.nn.functional as F # pytorch 激活函数的类
from torch import nn,optim # 构建模型和优化器

# 构建分类模型
class TextRNN(nn.Module):
    def __init__(self, parameter):
        super(TextRNN, self).__init__()
        embedding_dim = parameter['embedding_dim']
        hidden_size = parameter['hidden_size']
        output_size = parameter['output_size']
        num_layers = parameter['num_layers']
        dropout = parameter['dropout']
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, bidirectional=True, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size*2, output_size)
        
    def forward(self, x):
        out,(h, c)= self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
import os
import shutil
import pickle as pk
from torch.utils.tensorboard import SummaryWriter

# 记录日志
shutil.rmtree('textrnn') if os.path.exists('textrnn') else 1
writer = SummaryWriter('./textrnn', comment='textrnn')

# 构建模型
model = TextRNN(parameter).to(parameter['cuda'])

# 确定训练模式
model.train()

# 确定优化器和损失
optimizer = torch.optim.SGD(model.parameters(),lr=0.1, momentum=0.95, nesterov=True)
# optimizer = torch.optim.Adam(model.parameters(),lr = parameter['lr'], \
#                              weight_decay = 0.01)
criterion = nn.CrossEntropyLoss()

# 保存图
train_yield = batch_yield(train_chars,train_labels,parameter)
seqs,label,keys,epoch = next(train_yield)
writer.add_graph(model, (seqs,))

# 准备迭代器
train_yield = batch_yield(train_chars,train_labels,parameter)


# 开始训练
loss_cal = []
min_loss = float('inf')
with writer:
    while 1:
        seqs,label,keys,epoch = next(train_yield)
        if not keys:
            break
        out = model(seqs)
        loss = criterion(out, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_cal.append(loss.item())
        if epoch is not None:
            if (epoch+1)%1 == 0:
                loss_cal = sum(loss_cal)/len(loss_cal)
                if loss_cal < min_loss:
                    min_loss = loss_cal
                    torch.save(model.state_dict(), parameter['model_path']+'model-rnn.h5')
                print('epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, \
                                                       parameter['epoch'],loss_cal))
            writer.add_scalar('loss',loss_cal,global_step=epoch+1)
            loss_cal = [loss.item()]
    writer.flush()
    writer.close()
    
# torch.save(model.state_dict(), 'model-rnn.h5')

 38%|█████████████████████████████▎                                                | 200/532 [00:00<00:00, 1015.21it/s]

epoch [1/300], Loss: 1.0795


 19%|██████████████▊                                                                | 100/532 [00:00<00:00, 892.86it/s]

epoch [2/300], Loss: 1.0355


 19%|██████████████▊                                                                | 100/532 [00:00<00:00, 980.04it/s]

...

 19%|██████████████▊                                                                | 100/532 [00:00<00:00, 952.36it/s]

epoch [60/300], Loss: 0.0101


 19%|██████████████▊                                                                | 100/532 [00:00<00:00, 934.58it/s]

epoch [61/300], Loss: 0.0003


100%|███████████████████████████████████████████████████████████████████████████████| 532/532 [00:00<00:00, 953.41it/s]

# intent1模型训练（建议重启下进程）

In [1]:
# hava a look version
import torch
# import tensorflow as tf
# torch.__version__,tf.__version__
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')#torch.device('cpu')#
print(device,torch.__version__)
# print(device,torch.__version__,tf.__version__)

cpu 1.8.0+cpu


In [3]:
import os
import time
import gensim
import random
import numpy as np
import pickle as pk
import pandas as pd
from tqdm import tqdm
from operator import itemgetter
from collections import defaultdict
# from sklearn.cross_validation import train_test_split

parameter = {
    'min_count_word':1,
    'char2ind':None,
    'ind2char':None,
    'ind2embeding':None,
    'output_size': None,
    'epoch':500,
    'batch_size':100,
    'embedding_dim':300,
    'hidden_size':128,
    'num_layers':2, #堆叠LSTM的层数，默认值为1
    'dropout':0.5,
    'cuda':device,
    'lr':0.001,
    'num_unknow':0
}

def build_dataSet(data,parameter):
    # 构建训练集（chars，labels），构建词汇表（char2ind，ind2char）
    chars = []
    labels = []
    vocab = defaultdict(int)
    vocab['<pad>'] = parameter['min_count_word']
    for text,label in tqdm(zip(data.text,data.label)):
        chars.append(text)
        labels.append(label)
        for char in chars[-1]:
            vocab[char] += 1
    vocab['<unk>'] = parameter['min_count_word']
    for char in vocab:
        if vocab[char] < parameter['min_count_word']:
            del vocab[char]
    char2ind,ind2char = dict(zip(vocab.keys(),range(len(vocab)))), \
    dict(zip(range(len(vocab)),vocab.keys()))
    ind2embeding = np.random.randn(len(vocab), parameter['embedding_dim']).astype(np.float32) / np.sqrt(len(vocab))
    # 加载词向量
    w2v = gensim.models.Word2Vec.load('data/wiki.Mode')
    for ind,i in enumerate(char2ind.keys()):
        try:
            embedding = np.asarray(w2v.wv[i], dtype='float32')
            ind2embeding[ind] = embedding
        except:
            parameter['num_unknow'] += 1
    parameter['ind2char'] = ind2char
    parameter['char2ind'] = char2ind
    parameter['ind2embeding'] = ind2embeding
    parameter['output_size'] = len(set(labels))
    return np.array(chars),np.array(labels)

def batch_yield(chars,labels,parameter,shuffle = True):
    for train_epoch in range(parameter['epoch']):
        if shuffle:
            permutation = np.random.permutation(len(chars))
            chars = chars[permutation]
            labels = labels[permutation]
        max_len = 0
        batch_x,batch_y = [],[]
        for iters in tqdm(range(len(chars))):
            batch_ids = itemgetter(*chars[iters])(parameter['char2ind'])
            try:
                batch_ids = list(batch_ids)
            except:
                batch_ids = [batch_ids,0]
            if len(batch_ids) > max_len:
                max_len = len(batch_ids)
            batch_x.append(batch_ids)
            batch_y.append(labels[iters])
            if len(batch_x) >= parameter['batch_size']:
                batch_x = [np.array(list(itemgetter(*x_ids)(parameter['ind2embeding']))+[parameter['ind2embeding'][0]]*(max_len-len(x_ids))) for x_ids in batch_x]
                device = parameter['cuda']
                yield torch.from_numpy(np.array(batch_x)).to(device),torch.from_numpy(np.array(batch_y)).to(device).long(),True,None
                max_len,batch_x,batch_y = 0,[],[]
        batch_x = [np.array(list(itemgetter(*x_ids)(parameter['ind2embeding']))+[parameter['ind2embeding'][0]]*(max_len-len(x_ids))) for x_ids in batch_x]
        device = parameter['cuda']
        yield torch.from_numpy(np.array(batch_x)).to(device),torch.from_numpy(np.array(batch_y)).to(device).long(),True,train_epoch
        max_len,batch_x,batch_y = 0,[],[]
    yield None,None,False,None
    
def batch_yield_predict(chars,parameter):
    batch_x,batch_y = [],[]
    for iters in range(len(chars)):
        if chars[iters] in parameter['char2ind']:
            batch_x.append(parameter['ind2embeding'][parameter['char2ind'][chars[iters]]])
        else:
            batch_x.append(parameter['ind2embeding'][parameter['char2ind']['<unk>']])
    batch_x = [batch_x]
    device = parameter['cuda']
    return torch.from_numpy(np.array(batch_x)).to(device)#,torch.from_numpy(np.array(batch_y)).to(device).long()

[x_intent,y_intent,x_ner,y_ner] = pk.load(open('data/data-intent1-ner.pkl','rb'))
data = pd.DataFrame(np.array([x_intent,y_intent]).transpose())
data.columns = ['text','label']
train_chars,train_labels = build_dataSet(data,parameter)

train_yield = batch_yield(train_chars,train_labels,parameter)
seqs,label,keys,epoch = next(train_yield)
seqs,labels,_,_ = next(train_yield)
parameter['model_path'] = 'model/intent1/'
os.mkdir(parameter['model_path']) if not os.path.exists(parameter['model_path']) else 1

pk.dump(parameter,open(parameter['model_path']+'parameter.pkl','wb'))
seqs.shape,parameter['num_unknow']

D:\Anaconda3-5.3\lib\site-packages\ipykernel_launcher.py:103: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
203it [00:00, ?it/s]
D:\Anaconda3-5.3\lib\site-packages\ipykernel_launcher.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|                                                                                          | 0/203 [00:00<?, ?it/s]

(torch.Size([100, 16, 300]), 9)

In [4]:
# 校验一下
parameter['ind2char'] 
parameter['char2ind'] 
parameter['ind2embeding'].shape
parameter['output_size'],parameter['model_path']

(18, 'model/intent1/')

In [6]:
import torch.nn.functional as F # pytorch 激活函数的类
from torch import nn,optim # 构建模型和优化器

# 构建分类模型
class TextRNN(nn.Module):
    def __init__(self, parameter):
        super(TextRNN, self).__init__()
        embedding_dim = parameter['embedding_dim']
        hidden_size = parameter['hidden_size']
        output_size = parameter['output_size']
        num_layers = parameter['num_layers']
        dropout = parameter['dropout']
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, bidirectional=True, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size*2, output_size)
        
    def forward(self, x):
        out,(h, c)= self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

In [7]:
import os
import shutil
import pickle as pk
from torch.utils.tensorboard import SummaryWriter

# 记录日志
shutil.rmtree('textrnn') if os.path.exists('textrnn') else 1
writer = SummaryWriter('./textrnn', comment='textrnn')

# 构建模型
model = TextRNN(parameter).to(parameter['cuda'])

# 确定训练模式
model.train()

# 确定优化器和损失
optimizer = torch.optim.SGD(model.parameters(),lr=0.1, momentum=0.95, nesterov=True)
# optimizer = torch.optim.Adam(model.parameters(),lr = parameter['lr'], \
#                              weight_decay = 0.01)
criterion = nn.CrossEntropyLoss()

# 保存图
train_yield = batch_yield(train_chars,train_labels,parameter)
seqs,label,keys,epoch = next(train_yield)
writer.add_graph(model, (seqs,))

# 准备迭代器
train_yield = batch_yield(train_chars,train_labels,parameter)


# 开始训练
loss_cal = []
min_loss = float('inf')
with writer:
    while 1:
        seqs,label,keys,epoch = next(train_yield)
        if not keys:
            break
        out = model(seqs)
        loss = criterion(out, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_cal.append(loss.item())
        if epoch is not None:
            if (epoch+1)%1 == 0:
                loss_cal = sum(loss_cal)/len(loss_cal)
                if loss_cal < min_loss:
                    min_loss = loss_cal
                    torch.save(model.state_dict(), parameter['model_path']+'model-rnn.h5')
                print('epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, \
                                                       parameter['epoch'],loss_cal))
            writer.add_scalar('loss',loss_cal,global_step=epoch+1)
            loss_cal = [loss.item()]
    writer.flush()
    writer.close()
    
# torch.save(model.state_dict(), 'model-rnn.h5')

 49%|██████████████████████████████████████▉                                        | 100/203 [00:00<00:00, 775.17it/s]

epoch [1/500], Loss: 2.8909


 49%|██████████████████████████████████████▉                                        | 100/203 [00:00<00:00, 961.52it/s]

epoch [2/500], Loss: 2.8112


  0%|                                                                                          | 0/203 [00:00<?, ?it/s]

...

epoch [498/500], Loss: 0.5012


 49%|██████████████████████████████████████▉                                        | 100/203 [00:00<00:00, 990.08it/s]

epoch [499/500], Loss: 0.7995


100%|███████████████████████████████████████████████████████████████████████████████| 203/203 [00:00<00:00, 979.58it/s]

epoch [500/500], Loss: 0.7448
